In [ ]:
%cd D:/uitds

In [ ]:
import json

train_json = "data/json/vimmsd-train.json"

with open(train_json) as f:
    data = json.load(f)

In [ ]:
emojies  = []

for i in range(len(data)):
    emojies.append(data[str(i)]['caption'])

In [ ]:
import emoji

def extract_emojis(text_list):
    emoji_set = set()
    
    for text in text_list:
        # Find all emojis in each text and add them to the set
        for char in text:
            if char in emoji.EMOJI_DATA:
                emoji_set.add(char)
    
    return emoji_set

set_emoji = extract_emojis(emojies)

In [ ]:
print(*set_emoji)

In [ ]:
train_json = "data/json/vimmsd-public-test.json"

with open(train_json) as f:
    data = json.load(f)

In [ ]:
emojies  = []

for i in range(len(data)):
    emojies.append(data[str(i)]['caption'])

In [ ]:
test_set_emoji = extract_emojis(emojies)

In [ ]:
emoji_list = list(set_emoji.union(test_set_emoji))

In [ ]:
import emoji
from deep_translator import GoogleTranslator
import time

def extract_emojis_with_vietnamese_names(text_list):
    translator = GoogleTranslator(source='en', target='vi')
    emoji_dict = {}

    for text in text_list:
        for char in text:
            if char in emoji.EMOJI_DATA and char not in emoji_dict:
                # Get the English name of the emoji
                english_name = emoji.demojize(char).replace(":", "").replace("_", " ").title()
                
                try:
                    # Translate English name to Vietnamese
                    vietnamese_name = translator.translate(english_name)
                    time.sleep(0.5)  # To respect rate limits
                    emoji_dict[char] = vietnamese_name
                except Exception as e:
                    print(f"Translation error for '{english_name}': {str(e)}")
                    emoji_dict[char] = english_name  # Use English name as fallback

    return emoji_dict


unique_emojis_with_names = extract_emojis_with_vietnamese_names(emoji_list)
print(unique_emojis_with_names)

In [ ]:
unique_emojis_with_names

In [ ]:
error = ["Backhand Index Pointing Down", "Pensive Face", "Hamsa", "Play Button", "Cloud", "Lion", "Mango", "Rose","Leopard"]

In [ ]:
error_unique_emojis_with_names = {k: v for k, v in unique_emojis_with_names.items() if v in error}

In [ ]:
error_map_emoji = extract_emojis_with_vietnamese_names(error_unique_emojis_with_names.keys())

In [ ]:
# replace the error in the unique_emojis_with_names

for k, v in error_map_emoji.items():
    unique_emojis_with_names[k] = v

In [ ]:
unique_emojis_with_names

In [ ]:
# save to json emoji_vietnamese.json

with open("data/json/emoji_vietnamese.json", "w") as f:
    json.dump(unique_emojis_with_names, f, indent=4, ensure_ascii=False)

In [ ]:
captions  = []
labels = []
images = []
for i in range(len(data)):
    captions.append(data[str(i)]['caption'])
    labels.append(data[str(i)]['label'])
    images.append(data[str(i)]['image'])

In [ ]:
import re

def find_words_with_dots(text):
    # Regex pattern to match words with dots in the middle
    pattern = r'\b\w+(?:\.\w+)+\b'
    matches = re.findall(pattern, text)
    return matches

# Example usage:
text = 'Hôm nay tôi thấy một người muốn g.i.ế.t một c.hó.'
# Output: ['g.i.ế.t', 'c.hó']

for caption in captions:
    finding = find_words_with_dots(caption)
    if len(finding) > 0:
        print(finding)

In [ ]:
import re

def remove_dots_from_words(text: str) -> str:
    """
    Remove dots within words to reveal the original word.
    For example, 'g.i.ế.t' becomes 'giết' and 'c.hó' becomes 'chó'.
    Do not remove dots between numbers like '10.11.2005'.
    """
    # Use regex to find dots between letters only
    pattern = r'(?<=[^\W\d_])\.(?=[^\W\d_])'
    return re.sub(pattern, '', text, flags=re.UNICODE)

# Example usage
sample_text = "Hôm nay tôi thấy một người muốn g.i.ế.t một c.hó. Sinh ngày 10.11.2005"
clean_text = remove_dots_from_words(sample_text)
print(clean_text)


In [ ]:
import re

def remove_slashs_from_words(text: str) -> str:
    """
    Remove dots within words to reveal the original word.
    For example, 'g.i.ế.t' becomes 'giết' and 'c.hó' becomes 'chó'.
    Do not remove dots between numbers like '10.11.2005'.
    """
    # Use regex to find dots between letters only
    pattern = r'(?<=[^\W\d_])\/(?=[^\W\d_])'
    return re.sub(pattern, '', text, flags=re.UNICODE)

# Example usage
sample_text = "Hôm nay tôi thấy một người muốn g/i/ế/t một c/hó/ Sinh ngày 10/11/2005"
clean_text = remove_slashs_from_words(sample_text)
print(clean_text)

In [ ]:
import re

def remove_commas_from_words(text: str) -> str:
    """
    Remove dots within words to reveal the original word.
    For example, 'g.i.ế.t' becomes 'giết' and 'c.hó' becomes 'chó'.
    Do not remove dots between numbers like '10.11.2005'.
    """
    # Use regex to find dots between letters only
    pattern = r'(?<=[^\W\d_])\,(?=[^\W\d_])'
    return re.sub(pattern, '', text, flags=re.UNICODE)

# Example usage
sample_text = "Hôm nay tôi thấy một người muốn g,i,ế,t một c,hó, Sinh ngày 10,11,2005"
clean_text = remove_commas_from_words(sample_text)
print(clean_text)

In [ ]:
import re

def replace_emoticons_sad(text: str, replacement = " <icon> mặt buồn </icon> ") -> str:
    """
    Detects and replaces emoticons like :)), =)), :((, =(( with a custom replacement string.
    Emoticons can have any number of closing or opening parentheses after : or =.
    
    Args:
        text (str): The input text containing emoticons.
        replacement (str): The string to replace the emoticons with.
    
    Returns:
        str: The text with emoticons replaced by the replacement string.
    """
    # Regex pattern to match emoticons like :)), =)), :((, =((, :)))) and =(((, etc.
    pattern = r'(:|=)(\(+)'

    # Replace all matches with the replacement string
    result = re.sub(pattern, replacement, text)
    
    return result
def replace_emoticons_smile(text: str, replacement = " <icon> mặt cười </icon> ") -> str:
    """
    Detects and replaces emoticons like :)), =)), :((, =(( with a custom replacement string.
    Emoticons can have any number of closing or opening parentheses after : or =.
    
    Args:
        text (str): The input text containing emoticons.
        replacement (str): The string to replace the emoticons with.
    
    Returns:
        str: The text with emoticons replaced by the replacement string.
    """
    # Regex pattern to match emoticons like :)), =)), :((, =((, :)))) and =(((, etc.
    pattern = r'(:|=)(\)+)'

    # Replace all matches with the replacement string
    result = re.sub(pattern, replacement, text)
    
    return result
# Example usage
sample_text = "Hello :)) How are you? I'm feeling a bit :((( =(( but also =)) :)))"
new_text = replace_emoticons_sad(sample_text)
new_text = replace_emoticons_smile(new_text)
print(new_text)


In [ ]:
# open the emoji_vietnamese.json
with open("data/json/emoji_vietnamese.json") as f:
    emoji_vietnamese = json.load(f)


In [ ]:
with open("data/json/vimmsd-train.json") as f:
    data = json.load(f)

captions  = []
labels = []
images = []

for i in range(len(data)):
    captions.append(data[str(i)]['caption'])
    labels.append(data[str(i)]['label'])
    images.append(data[str(i)]['image'])


In [ ]:
newcaptions = []
for caption in captions:
    # remove dot and slash
    caption = remove_dots_from_words(caption)
    caption = remove_slashs_from_words(caption)
    caption = remove_commas_from_words(caption)
    # replace emoticons
    caption= replace_emoticons_sad(caption)
    caption= replace_emoticons_smile(caption)
    # replace emoji
    for k, v in emoji_vietnamese.items():
        caption = caption.replace(k,f" <icon> {v} </icon> ")
    newcaptions.append(caption)

In [ ]:
# save to a new json file

new_data = {}
for i in range(len(data)):
    new_data[str(i)] = {'caption': newcaptions[i], 'label': labels[i], 'image':images[i]}

In [ ]:
with open("data/json/vimmsd-train-preprocess.json", "w") as f:
    json.dump(new_data, f, indent=4, ensure_ascii=False)

In [ ]:
with open("data/json/vimmsd-public-test.json") as f:
    data = json.load(f)

captions  = []
labels = []
images = []

for i in range(len(data)):
    captions.append(data[str(i)]['caption'])
    labels.append(data[str(i)]['label'])
    images.append(data[str(i)]['image'])

newcaptions = []
for caption in captions:
    # remove dot and slash
    caption = remove_dots_from_words(caption)
    caption = remove_slashs_from_words(caption)
    caption = remove_commas_from_words(caption)
    # replace emoticons
    caption= replace_emoticons_sad(caption)
    caption= replace_emoticons_smile(caption)
    # replace emoji
    for k, v in emoji_vietnamese.items():
        caption = caption.replace(k,f" <icon> {v} </icon> ")
    newcaptions.append(caption)
    
new_data = {}
for i in range(len(data)):
    new_data[str(i)] = {'caption': newcaptions[i], 'label': labels[i], 'image':images[i]}
    
with open("data/json/vimmsd-public-test-preprocess.json", "w") as f:
    json.dump(new_data, f, indent=4, ensure_ascii=False)
    